In [1]:
import polars as pl
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection, connections, utility
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import torch
import time

/opt/conda/envs/milvus/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Milvus server setup

connections.connect(
    alias="default",
    host="192.168.0.133",
    port="19530"
)

In [3]:
MODEL_NAME = "jhgan/ko-sbert-multitask"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(MODEL_NAME, device=device)

In [4]:
# embedding data load

df = pl.read_parquet("train_with_sbert_embedding.parquet")

In [6]:
df.head()

id,document,label,embedding
i64,str,i64,list[f64]
8112052,"""어릴때보고 지금다시봐도 재밌어요ㅋㅋ""",1,"[0.020148, 0.002106, … 0.032176]"
8132799,"""디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 …",1,"[0.0021, 0.043697, … 0.091742]"
4655635,"""폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음…",1,"[-0.003628, 0.06392, … 0.068882]"
9251303,"""와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데…",1,"[0.045442, 0.018689, … 0.021112]"
10067386,"""안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.""",1,"[0.037311, 0.004324, … -0.020018]"


In [7]:
# Embedding data dimention
dim = len(df["embedding"][0])

if utility.has_collection("reviews"):
    utility.drop_collection("reviews")

# Field Schema
id_field = FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False)
vec_field = FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=dim)

# Collection Schema
schema = CollectionSchema(fields=[id_field, vec_field], dscription="review embedding")
col = Collection(name="reviews", schema=schema)

# Dataframe list형으로 변환
ids = df['id'].to_list()
vecs = df['embedding'].to_list()

In [8]:
# Batch insert

batch_size = 10000
batch_num = 0
n = len(ids)
batch_max = n//batch_size

for i in tqdm(range(0,n,batch_size)):
    batch_ids = ids[i:i+batch_size]
    batch_vecs = vecs[i:i+batch_size]
    col.insert([batch_ids, batch_vecs])
col.flush()

100%|██████████| 20/20 [00:29<00:00,  1.46s/it]


In [16]:
def encode_with_skovert(text: str) -> list[float]:
    embedding = model.encode(text)
    return embedding.tolist()

In [22]:
index_configs = {
    "GPU_BRUTE_FORCE": {
        "index_type": "GPU_BRUTE_FORCE",
        "metric_type": "IP",
        "params": {}
    },
    "GPU_IVF_FLAT": {
        "index_type": "GPU_IVF_FLAT",
        "metric_type": "IP",
        "params": {"nlist": 1024}
    },
    "GPU_IVF_PQ": {
        "index_type": "GPU_IVF_PQ",
        "metric_type": "IP",
        "params": {
            "nlist" : 1024,
            "m": 16,
            "nbits": 8
        }
    },
    "HNSW": {
        "index_type": "HNSW",
        "metric_type": "IP",
        "params": {
            "M": 32,
            "efConstruction": 200
        }
    },
    "GPU_CAGRA": {
        "index_type": "GPU_CAGRA",
        "metric_type": "IP",
        "params": {
            "graph_degree":32,
            "intermediate_graph_degree": 64
        }
    }
}

In [ ]:
def search_with_index(index_name: str, user_review: str, top_k: int=5):
    cfg = index_configs[index_name]
    print('Initializing Start')
    try:
        col.release()
        col.drop_index()
    except Exception as e:
        print("drop_index error", e)

    col.create_index(
        field_name="embedding",
        index_params={
            "index_type": cfg["index_type"],
            "metric_type": cfg["metric_type"],
            "params": cfg["params"]
        }
    )

    col.load()
    print('Initializing Done')
    print()
    print('Input Encoding Start')
    query_vec = encode_with_skovert(user_review)
    print('Input Encoding Done')
    
    if cfg["index_type"] in ("GPU_IVF_FLAT", "IVF_FLAT", "GPU_IVF_PQ", "IVF_PQ"):
        search_params = {"nprobe": 16}
    elif cfg["index_type"] == "HNSW":
        search_params = {"ef": 64}
    else:
        search_params = {}

    start_time = time.time()
    print(f'{index_name} indexing start...{start_time}')
    results = col.search(
        data=[query_vec],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        output_fields=["id"]
    )
    elapsed = time.time() - start_time

    hits = results[0]

    ids = [hit.id for hit in hits]

    top_df = df.filter(pl.col("id").is_in(ids))

    id_to_rank = {id_: rank for rank, id_ in enumerate(ids)}
    top_df = top_df.with_columns(
        pl.col("id").map_elements(lambda x: id_to_rank.get(x, 10**9)).alias("_rank")
    ).sort("_rank").drop("_rank")

    return {
        "index_type" : index_name,
        "elapsed_sec" : elapsed,
        "hits" : hits,
        "top_df" : top_df
    }

In [39]:
user_review = "와 이 영화 진짜 재밌다. 특히 마지막 반전이 미쳤음"

results_all = {}

for name in ["GPU_BRUTE_FORCE", "GPU_IVF_FLAT", "GPU_IVF_PQ", "HNSW"]:
    print(f"=== {name} ===")
    res = search_with_index(name, user_review, top_k=5)
    results_all[name] = res

    print(f"검색 시간: {res['elapsed_sec']:.4f} 초")
    print(res["top_df"].select(["id", "document"]))
    print()

=== GPU_BRUTE_FORCE ===
검색 시간: 0.8335 초
shape: (5, 2)
┌─────────┬─────────────────────────────────┐
│ id      ┆ document                        │
│ ---     ┆ ---                             │
│ i64     ┆ str                             │
╞═════════╪═════════════════════════════════╡
│ 5150926 ┆ 반전이있는 영화였네요..최고     │
│ 2026178 ┆ 아름다운영화,마지막 반전에서 더 │
│         ┆ 큰 감동이 오는 영화,            │
│ 1088084 ┆ 아~정말 재미있게 본영화 마지막  │
│         ┆ 반전에~^^                       │
│ 4591261 ┆ 마지막까지 반전이나온영화       │
│         ┆ 재미있게 봤어요                 │
│ 4597497 ┆ 마지막 반전에 사람을 깜짝       │
│         ┆ 놀라게 했던 영화. 최고의 …      │
└─────────┴─────────────────────────────────┘

=== GPU_IVF_FLAT ===
검색 시간: 0.0602 초
shape: (5, 2)
┌─────────┬─────────────────────────────────┐
│ id      ┆ document                        │
│ ---     ┆ ---                             │
│ i64     ┆ str                             │
╞═════════╪═════════════════════════════════╡
│ 5150926 ┆ 반전이있는 영화였네요..최고     │
│ 2026178 ┆ 아름다운영화,마지막 반전에서 더 